In [1]:
import pandas as pd 
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from scipy import optimize
import copy
import itertools
import math
from sklearn.svm import SVC
from sklearn.utils import shuffle
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('compas-scores-two-years.csv')

In [3]:
# data preprocessing

data.dropna()
data_subset = data[['sex','age','race', 'juv_misd_count', 'juv_fel_count','priors_count', 'decile_score','two_year_recid']]
data_subset["two_year_recid"] = data["two_year_recid"].apply(lambda x: -1 if x==0 else 1)

#selecting race Caucusian / African American and encoding to 0/1

data_subset = data_subset[(data_subset["race"]=='Caucasian') |(data_subset["race"]=='African-American') ]
data_subset["race_cat"] = data_subset["race"].apply(lambda x: 1 if x == "Caucasian" else 0)
data_subset = data_subset.drop(columns = "race")

# encoding gender to 0/1

data_subset["gender_cat"] = data_subset["sex"].apply(lambda x: 1 if x == "Female" else 0)
data_subset = data_subset.drop(columns = "sex")

y = data_subset["two_year_recid"]
protected = data_subset["race_cat"]
data2 = data_subset.drop(columns=["two_year_recid","race_cat"])

y, protected_attr, data2 = shuffle(y, protected, data2, random_state = 0)

<ipython-input-3-a5eb1a764a8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset["two_year_recid"] = data["two_year_recid"].apply(lambda x: -1 if x==0 else 1)


In [4]:
train = int(len(data2)*.80)
x_train, y_train, race_train = data2[:train], y[:train], protected[:train]
x_test, y_test, race_test = data2[train:], y[train:],protected[train:]

In [5]:
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
print(f"Logistic Regression Accuracy: {clf.score(x_train, y_train)}")
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(x_train))

Logistic Regression Accuracy: 0.6752032520325203


In [6]:
print(f"Logistic Regression Accuracy: {clf.score(x_test, y_test)}")

Logistic Regression Accuracy: 0.6829268292682927


In [7]:
def p_rule(race, predicted_y):
    
    not_protected = np.where(race != 1)[0]
    protected = np.where(race == 1)[0] 
    protected_preds = np.where(predicted_y[protected] == 1)
    nonpro_preds = np.where(predicted_y[not_protected] == 1)
    protected_perc = (protected_preds[0].shape[0]/protected.shape[0])
    nonpro_perc = (nonpro_preds[0].shape[0]/not_protected.shape[0])
    perc_ratio = protected_perc/nonpro_perc
    return perc_ratio, protected_perc, nonpro_perc

In [8]:
print(f"P-rule: {p_rule(race_train, clf.predict(x_train))}")

P-rule: (1.0074129744132818, 0.410958904109589, 0.40793489318413023)


In [9]:
p_rule(race_test, clf.predict(x_test))

(0.9684216636674987, 0.4161490683229814, 0.42971887550200805)